# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제

In [3]:
pd_data.iloc[:,0]

0           youth
1           youth
2     middle_aged
3          senior
4          senior
5          senior
6     middle_aged
7           youth
8           youth
9          senior
10          youth
11    middle_aged
12    middle_aged
13         senior
Name: age, dtype: object

In [4]:
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree.export import export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
import warnings
warnings.filterwarnings(action='ignore')

C:\Users\jinji\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.tree.export module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
def get_gini(df, label):
   
    # 타겟변수의 gini 구하기
    target_class = list(df[label].unique())
    
    target_yes = len(df[df[label]==target_class[0]])
    target_no = len(df[df[label]!=target_class[0]])
    p = target_yes/len(df[label])
    q = 1-p
    
    gini = 1 - p**2-q**2
    
    return  gini

In [7]:
target_class = list(pd_data["class_buys_computer"].unique())
target_class

['no', 'yes']

In [8]:
get_gini(pd_data,'class_buys_computer')     # 답과 같이 나옴

0.4591836734693879

In [9]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693879

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [10]:
# 원리
"""
def get_binary_split(df, attribute):
    
    # 변수의 클래스를 pool로 받음
    pool = df[attribute].unique()     
    pool1 = list(map(list,itertools.combinations(pool,1)))
    pool2 = list(map(list,itertools.combinations(pool,2)))
    #nC1과 nC2
    
    result = pool1 + pool2

    return result 
"""

'\ndef get_binary_split(df, attribute):\n    \n    # 변수의 클래스를 pool로 받음\n    pool = df[attribute].unique()     \n    pool1 = list(map(list,itertools.combinations(pool,1)))\n    pool2 = list(map(list,itertools.combinations(pool,2)))\n    #nC1과 nC2\n    \n    result = pool1 + pool2\n\n    return result \n'

In [118]:
import itertools # 변수의 모든 클래스 조합을 얻기 위해 itertools 불러오기

# 클래스가 4개 이상일 때

def get_binary_split(df,attribute):
    
    pool = df[attribute].unique()     # 변수의 클래스를 pool로 받음
    result = list()
    
    for i in range(0,len(pool)+1):
        result.extend(list(map(list,itertools.combinations(pool,i))))
    
    return result[1:-1]


In [17]:
get_binary_split(pd_data, "age")    # 답과 같이 나옴

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

In [18]:
get_binary_split(pd_data, "age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

In [19]:
# 클래스 4개 이상일 때 
test = np.array(['A', 'B', 'C','D'])
pool = test     # 변수의 클래스를 pool로 받음
result = list()
    
for i in range(0,len(pool)+1):
     result.extend(list(map(list,itertools.combinations(pool,i))))
result[1:-1]

[['A'],
 ['B'],
 ['C'],
 ['D'],
 ['A', 'B'],
 ['A', 'C'],
 ['A', 'D'],
 ['B', 'C'],
 ['B', 'D'],
 ['C', 'D'],
 ['A', 'B', 'C'],
 ['A', 'B', 'D'],
 ['A', 'C', 'D'],
 ['B', 'C', 'D']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.
- 결과로 나온 Dictionary의 Key 값은 해당 class 들로 이루어진 tuple 형태로 들어가 있습니다.

In [25]:
get_binary_split(pd_data, "age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

In [151]:
def get_attribute_gini_index(df, attribute, label):
    # get_attribute_gini_index(pd_data, "age", "class_buys_computer")
    
    class_values = list(df[attribute].unique())    # ('youth','middle_age','senior')
    key = get_binary_split(df,attribute)
    gini = list()
    
    for i in range(0,len(class_values)):
        D1 = len(df[df[attribute]==class_values[i]])/len(df)
        D2 = 1-D1
        # get_gini도 이용해서 gini 계산
        gini = D1 * get_gini(df[df[attribute]==class_values[i]],label) + D2 * get_gini(df[df[attribute]!=class_values[i]],label) 
        print(gini)
    
    result = dict(zip(key,gini)) 
    
    return result

In [152]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

0.3936507936507936
0.35714285714285715
0.457142857142857


TypeError: zip argument #2 must support iteration

# 해결하지 못함
1. get_binary_split을 이용해서 지니를 구하는 함수를 못만들겠습니다.  
저 함수의 결과값을 인자로 사용 못하겠음  
2. 연결되서 딕셔너리값에 저장하지 못함.  

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

결과를 확인해보도록 하겠습니다.

In [ ]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [148]:
##문제1 답안
print(get_binary_split(pd_data, "income"))
print(get_attribute_gini_index(pd_data, "income", "class_buys_computer"))

[['high'], ['medium'], ['low'], ['high', 'medium'], ['high', 'low'], ['medium', 'low']]
0.44285714285714284
0.4583333333333333
0.45


TypeError: 'float' object is not iterable

In [ ]:
##문제2 답안



In [ ]:
##문제3 답안

